In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from leafmap.common import evaluate_model, plot_actual_vs_predicted, download_file

In [2]:
zhvi_file= "/home/zyang91/Desktop/data/zillow_home_value_index_by_zipcode.csv"

In [3]:
zhvi_df= pd.read_csv(zhvi_file,dtype={'RegionName': str})
zhvi_df.index= zhvi_df["RegionName"].apply(lambda x: f"zip/{x}")
zhvi_df

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31,2025-01-31
RegionName,,,,,,,,,,,,,,,,,,,,,
zip/77494,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,209050.47676,...,493768.79581,495075.41384,495380.40257,495002.98333,495408.56194,496096.30546,497215.16189,497931.31997,498348.02484,498770.07052
zip/8701,61148,2,8701,zip,NJ,NJ,Lakewood,"New York-Newark-Jersey City, NY-NJ-PA",Ocean County,129618.78060,...,567181.88952,574111.68852,579228.58585,583825.79895,588765.87771,594583.48573,599788.27507,603106.03147,605075.74933,605701.71314
zip/77449,91940,3,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,103655.52834,...,281613.08126,282147.02210,282107.28078,281872.89042,281691.18638,281502.99185,281086.41028,280298.90222,279570.22174,278960.08768
zip/11368,62080,4,11368,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,146323.41156,...,448237.17350,452805.95427,453500.45273,452997.37976,452371.99866,453320.49579,453075.34378,451863.57197,449627.30018,447532.07593
zip/77084,91733,5,77084,zip,TX,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris County,102106.86183,...,274112.51755,274613.19668,274446.91817,274155.37019,273883.45917,273631.25456,273125.76564,272528.81542,272190.92108,271949.29478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zip/14441,63527,39992,14441,zip,NY,NY,Dresden,"Rochester, NY",Yates County,95200.50699,...,207411.50108,212859.34574,217705.20709,220747.04046,223033.16078,225156.69128,227314.26864,228544.60812,228472.62049,227382.44448
zip/98628,99927,39992,98628,zip,WA,WA,Klickitat,NaN,Klickitat County,NaN,...,180804.48613,183691.42860,184378.49311,182270.09823,179856.66170,177621.28838,176436.41050,175243.01948,174289.08096,172769.70567
zip/52163,80861,39992,52163,zip,IA,IA,Protivin,NaN,Howard County,NaN,...,108590.53252,110531.05868,112546.78660,114074.31279,114923.34847,114988.32175,115144.24508,115385.62968,115883.51422,116227.22666


In [4]:
embeddings_file = "/home/zyang91/Desktop/us/zcta_embeddings.csv"

In [5]:
zipcode_embeddings = pd.read_csv(embeddings_file).set_index("place")
zipcode_embeddings

,state,county,city,population,latitude,longitude,feature0,feature1,feature2,feature3,...,feature320,feature321,feature322,feature323,feature324,feature325,feature326,feature327,feature328,feature329
place,,,,,,,,,,,,,,,,,,,,,
zip/97910,OR,Malheur County,Jordan Valley,609,42.749076,-117.511459,-0.138227,1.120377,0.072900,0.297442,...,-1.588909e-01,-0.168708,1.231994,-1.557645e-01,3.043214,-1.697491e-01,0.177463,-0.001661,-1.010470e-03,4.495589
zip/89412,NV,Washoe County,Gerlach,98,41.102934,-119.695361,-0.141379,1.422782,0.234269,0.159156,...,-1.574173e-01,-0.043606,2.788701,-6.254692e-02,3.700745,-1.698268e-01,-0.137990,-0.024385,-2.950600e-04,3.399393
zip/88030,NM,Luna County,Deming,24139,32.191634,-107.729431,-0.046666,1.414424,0.146803,1.113256,...,-6.540800e-04,0.437475,4.229295,2.291990e-01,2.098469,1.150497e+00,0.716122,-0.116499,-5.116325e-02,3.866543
zip/82633,WY,Converse County,Douglas,9478,43.022270,-105.410250,-0.090293,1.266280,0.447868,0.781861,...,-3.377113e-02,0.579775,2.688665,1.756692e-01,0.990921,1.644879e+00,0.222517,-0.047864,-4.237000e-05,7.453567
zip/59538,MT,Phillips County,Malta,2936,48.112019,-107.845520,-0.092886,1.256203,-0.050897,0.321954,...,-1.699154e-01,-0.088829,0.338914,-1.029619e-01,-0.156583,1.493696e+00,2.259007,-0.161916,-1.086910e-03,0.972243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zip/43117,OH,Pickaway County,Derby,69,39.768789,-83.206122,-0.027339,1.556422,1.229264,0.801588,...,-1.090080e-01,-0.039277,1.950797,-1.548614e-01,-0.163302,-1.699533e-01,1.940206,1.456303,-9.436400e-04,2.706647
zip/14302,NY,Niagara County,Niagara Falls,229,43.093896,-79.049193,-0.081384,1.241892,0.655743,0.431606,...,8.140615e-01,-0.000713,-0.117548,-7.834878e-02,1.176950,-1.900440e-02,0.445953,-0.071082,-3.293640e-03,1.193913
zip/10162,NY,New York County,New York,2268,40.769298,-73.949942,-0.130655,1.159695,3.368129,1.547286,...,-9.400000e-07,-0.130648,-0.000000,-3.996429e-02,-0.023050,-2.800000e-07,0.033541,-0.000002,-2.000000e-08,-0.097251


In [11]:
data= zhvi_df.join(zipcode_embeddings, how="inner")
data

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,feature320,feature321,feature322,feature323,feature324,feature325,feature326,feature327,feature328,feature329
zip/77494,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,209050.47676,...,-0.000001,-0.000573,-7.384640e-02,3.741291,6.835684,7.062005e+00,2.521905,1.494393,-0.000000e+00,0.663955
zip/77449,91940,3,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,103655.52834,...,-0.017497,-0.003990,-1.646668e-01,4.616399,6.331781,6.382573e+00,2.943955,1.990187,-4.000000e-06,1.199221
zip/11368,62080,4,11368,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,146323.41156,...,-0.099274,2.189546,-2.500000e-07,6.165315,-0.082513,-1.300000e-07,3.876507,-0.031210,-4.438200e-04,3.084769
zip/77084,91733,5,77084,zip,TX,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris County,102106.86183,...,-0.006604,-0.015875,-1.620866e-01,4.832126,6.007081,6.243350e+00,1.670585,2.623777,-7.000000e-08,0.925346
zip/79936,93144,6,79936,zip,TX,TX,El Paso,"El Paso, TX",El Paso County,88633.20026,...,2.817620,3.308279,1.424846e+00,5.052125,-0.169852,-1.595640e-03,0.101079,-0.168736,-4.000000e-08,11.867829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zip/14441,63527,39992,14441,zip,NY,NY,Dresden,"Rochester, NY",Yates County,95200.50699,...,4.180083,-0.131965,1.830823e+00,0.323147,0.010833,-9.629160e-03,1.642928,0.619273,-1.592139e-02,0.255313
zip/98628,99927,39992,98628,zip,WA,WA,Klickitat,NaN,Klickitat County,NaN,...,-0.167128,0.219471,-1.258920e-03,-0.017126,4.365533,4.186756e-01,0.186192,-0.163884,-7.537420e-03,1.982381
zip/52163,80861,39992,52163,zip,IA,IA,Protivin,NaN,Howard County,NaN,...,-0.160310,0.264810,-1.492518e-01,-0.099826,0.856826,-2.017140e-02,9.899288,-0.132805,-2.782500e-04,0.601016
zip/82515,93941,39992,82515,zip,WY,WY,Hudson,"Riverton, WY",Fremont County,NaN,...,0.308261,0.603739,3.155533e+00,-0.080326,1.294188,8.093709e-01,-0.147521,-0.047723,-1.597920e-03,6.474327


In [12]:
embedding_features= [f"feature{x}" for x in range(330)]
label= "2024-10-31"

In [13]:
data= data.dropna(subset=[label])
data

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,feature320,feature321,feature322,feature323,feature324,feature325,feature326,feature327,feature328,feature329
zip/77494,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,209050.47676,...,-0.000001,-0.000573,-7.384640e-02,3.741291,6.835684,7.062005e+00,2.521905,1.494393,-0.000000e+00,0.663955
zip/77449,91940,3,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,103655.52834,...,-0.017497,-0.003990,-1.646668e-01,4.616399,6.331781,6.382573e+00,2.943955,1.990187,-4.000000e-06,1.199221
zip/11368,62080,4,11368,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,146323.41156,...,-0.099274,2.189546,-2.500000e-07,6.165315,-0.082513,-1.300000e-07,3.876507,-0.031210,-4.438200e-04,3.084769
zip/77084,91733,5,77084,zip,TX,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris County,102106.86183,...,-0.006604,-0.015875,-1.620866e-01,4.832126,6.007081,6.243350e+00,1.670585,2.623777,-7.000000e-08,0.925346
zip/79936,93144,6,79936,zip,TX,TX,El Paso,"El Paso, TX",El Paso County,88633.20026,...,2.817620,3.308279,1.424846e+00,5.052125,-0.169852,-1.595640e-03,0.101079,-0.168736,-4.000000e-08,11.867829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zip/14441,63527,39992,14441,zip,NY,NY,Dresden,"Rochester, NY",Yates County,95200.50699,...,4.180083,-0.131965,1.830823e+00,0.323147,0.010833,-9.629160e-03,1.642928,0.619273,-1.592139e-02,0.255313
zip/98628,99927,39992,98628,zip,WA,WA,Klickitat,NaN,Klickitat County,NaN,...,-0.167128,0.219471,-1.258920e-03,-0.017126,4.365533,4.186756e-01,0.186192,-0.163884,-7.537420e-03,1.982381
zip/52163,80861,39992,52163,zip,IA,IA,Protivin,NaN,Howard County,NaN,...,-0.160310,0.264810,-1.492518e-01,-0.099826,0.856826,-2.017140e-02,9.899288,-0.132805,-2.782500e-04,0.601016
zip/82515,93941,39992,82515,zip,WY,WY,Hudson,"Riverton, WY",Fremont County,NaN,...,0.308261,0.603739,3.155533e+00,-0.080326,1.294188,8.093709e-01,-0.147521,-0.047723,-1.597920e-03,6.474327


In [14]:
data = data[embedding_features + [label]]
X = data[embedding_features]
y = data[label]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [15]:
# Initialize and train a simple linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

In [16]:
evaluation_df = pd.DataFrame({"y": y_test, "y_pred": y_pred})
metrics = evaluate_model(evaluation_df)
print(metrics)

{'r2': 0.7923863392335829, 'r': np.float64(0.890846915263462), 'rmse': 131888.38743343446, 'mae': 72573.83922121955, 'mape': 0.26673190601469865}


In [17]:
xy_lim = (0, 3_000_000)
plot_actual_vs_predicted(
    evaluation_df,
    xlim=xy_lim,
    ylim=xy_lim,
    title="Actual vs Predicted Home Values",
    x_label="Actual Home Value",
    y_label="Predicted Home Value",
)

In [18]:
evaluate_df= pd.DataFrame({"y": y_test, "y_pred": y_pred})
metrics = evaluate_model(evaluate_df)
print(metrics)

{'r2': 0.7923863392335829, 'r': np.float64(0.890846915263462), 'rmse': 131888.38743343446, 'mae': 72573.83922121955, 'mape': 0.26673190601469865}


In [19]:
evaluate_df.head()

,y,y_pred
zip/62943,162222.55793,204549.127895
zip/63653,72232.23067,23479.718488
zip/64117,199195.44126,156224.812811
zip/74931,150131.29134,243551.452823
zip/98168,558843.92154,494979.367988


In [20]:
xy_lim=(0,3000000)
plot_actual_vs_predicted(
    evaluate_df, x_label="Actual", y_label="Predicted", xlim= xy_lim, ylim=xy_lim,
    title="Actual vs Predicted ZHVI",
)

In [22]:
k=5
knn_model = KNeighborsRegressor(n_neighbors=k)
knn_model.fit(X_train, y_train)
y_pred= knn_model.predict(X_test)


In [23]:
plot_actual_vs_predicted(
    evaluate_df, x_label="Actual", y_label="Predicted", xlim= xy_lim, ylim=xy_lim,
    title=f"Actual vs Predicted ZHVI (KNN, k={k})",
)

In [24]:
evaluation_df= pd.DataFrame({"y": y_test, "y_pred": y_pred})
metrics = evaluate_model(evaluation_df)
print(metrics)

{'r2': 0.787670314194312, 'r': np.float64(0.8889091262880304), 'rmse': 133377.92408499756, 'mae': 60408.591463699224, 'mape': 0.2067017807110204}
